In [2]:
!pip install librosa matplotlib numpy scipy praat-parselmouth tensorflow-hub
!pip install fastapi uvicorn numpy matplotlib librosa parselmouth tensorflow tensorflow-hub python-multipart

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 87.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import librosa
import librosa.display
import parselmouth
from parselmouth.praat import call
import tensorflow as tf
import tensorflow_hub as hub
import csv
import warnings
from google.colab import files

warnings.filterwarnings('ignore')

# --- MODERN DESIGN PALETTE ---
C_BG = '#0B0F19'      # Deep Space Navy
C_HL = '#00E676'      # Neon Healthy Green
C_WARN = '#FFEA00'    # Neon Warning Yellow
C_CRIT = '#FF1744'    # Neon Critical Red
C_BLUE = '#2979FF'    # Electric Blue
C_PURP = '#D500F9'    # Lethargy Purple
C_TEXT = '#FFFFFF'

# Load YAMNet Engine
print("Waking up Guardian AI Engine...")
yamnet_model = hub.load('https://tfhub.dev/google/yamnet/1')
class_map_path = yamnet_model.class_map_path().numpy().decode('utf-8')
class_names = [row['display_name'] for row in csv.DictReader(open(class_map_path))]

def get_vocal_texture_safe(snd):
    try:
        pitch = call(snd, "To Pitch", 0.0, 75, 600)
        point_process = call([snd, pitch], "To PointProcess (cc)")
        jitter = call([snd, point_process], "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
        shimmer = call([snd, point_process], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
        if np.isnan(jitter) or jitter == 0: return 0.2, 1.0
        return jitter * 100, shimmer * 100
    except: return 0.15, 0.8

def calculate_modern_risk(f0_std, wpm, silence_ratio, cry_score, mumble_score, sigh_score, laugh_score):
    """
    Calculates risk based on the full Clinical Requirement Table.
    Includes Laughter as a 'Protective Factor' (reduces risk).
    """
    score = 10 # Baseline

    # 1. Pitch Flatness (Flat Affect)
    score += np.clip((30 - f0_std) * 1.5, 0, 30)

    # 2. Psychomotor Lethargy (Speed)
    score += np.clip((130 - wpm) * 0.4, 0, 30)

    # 3. Cognitive Load (Silence)
    score += np.clip(silence_ratio * 0.5, 0, 20)

    # 4. Crisis Markers (Crying = High Risk)
    score += np.clip(cry_score * 100, 0, 25)

    # 5. Lethargy Markers (Whispering/Mumbling)
    score += np.clip(mumble_score * 50, 0, 15)

    # 6. Anxiety Markers (Sighing/Breathing)
    score += np.clip(sigh_score * 50, 0, 10)

    # 7. Positive Engagement (Laughter reduces risk)
    score -= np.clip(laugh_score * 50, 0, 20)

    # Normalize 0-100
    final_pct = int(np.clip(score, 5, 100))

    if final_pct > 60: level, col = "CRITICAL ALERT", C_CRIT
    elif final_pct > 30: level, col = "MODERATE WARNING", C_WARN
    else: level, col = "STABLE / HEALTHY", C_HL
    return final_pct, level, col

def draw_medical_cockpit(file_path):
    # --- DATA PROCESSING ---
    y, sr = librosa.load(file_path, sr=16000)
    duration = librosa.get_duration(y=y, sr=sr)
    snd = parselmouth.Sound(file_path)

    # Physics Metrics
    pitch = snd.to_pitch()
    f_vals = pitch.selected_array['frequency']
    f_vals[f_vals == 0] = np.nan
    f0_std = np.nanstd(f_vals)
    if np.isnan(f0_std): f0_std = 12.0

    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    peaks = librosa.util.peak_pick(onset_env, pre_max=3, post_max=3, pre_avg=3, post_avg=5, delta=0.5, wait=10)
    wpm = (len(peaks) / duration) * 60

    intervals = librosa.effects.split(y, top_db=25)
    silence_ratio = ((duration - (sum([i[1]-i[0] for i in intervals])/sr)) / duration) * 100
    jitter, shimmer = get_vocal_texture_safe(snd)

    # AI YAMNet Scores
    scores, _, _ = yamnet_model(tf.cast(y, tf.float32))
    m_scores = np.mean(scores.numpy(), axis=0)

    def get_val(name): return m_scores[class_names.index(name)] if name in class_names else 0.0

    # --- GUARDIAN LABELS EXTRACTION ---
    speech_s = get_val('Speech')
    silence_s = get_val('Silence')
    sigh_s = get_val('Sigh') + get_val('Breathing') # Combined Anxiety
    mumble_s = get_val('Whispering') # Proxy for Lethargy
    cry_s = get_val('Crying, sobbing') # Crisis
    laugh_s = get_val('Laughter') # Positive

    # Risk Calculation
    risk_pct, risk_lvl, risk_col = calculate_modern_risk(f0_std, wpm, silence_ratio, cry_s, mumble_s, sigh_s, laugh_s)

    # --- UI BUILDER ---
    # Massive height to prevent overflow
    fig, axes = plt.subplots(nrows=10, ncols=1, figsize=(18, 95), facecolor=C_BG)
    plt.subplots_adjust(hspace=0.9, bottom=0.05)

    # ROW 0: HEADER
    axes[0].set_facecolor('#111827')
    axes[0].text(0.5, 0.5, "GUARDIAN AI: BIOMETRIC SCAN", color=C_TEXT, fontsize=32, ha='center', fontweight='black')
    axes[0].axis('off')

    # ROW 1: SPEED
    axes[1].barh([0], [200], color='#222', height=0.3)
    axes[1].barh([0], [wpm], color=C_BLUE, height=0.3)
    axes[1].set_title(f"1. PSYCHOMOTOR SPEED ({int(wpm)} WPM)", color=C_TEXT, fontsize=22, fontweight='bold', pad=20)
    axes[1].set_xlabel("WHAT IS THIS: Measures 'Psychomotor Retardation' via speech frequency.\nHOW IT HELPS: Depression slows cognitive processing; lower WPM equals clinical lethargy.", color=C_WARN, fontsize=16, labelpad=15); axes[1].axis('off')

    # ROW 2: PROSODY
    axes[2].plot(pitch.xs(), f_vals, color=C_HL, linewidth=3)
    axes[2].set_title(f"2. PROSODY CONTOUR (SD: {f0_std:.1f}Hz)", color=C_TEXT, fontsize=22, fontweight='bold', pad=20)
    axes[2].set_xlabel("WHAT IS THIS: Tracks the fundamental frequency variance (Vocal Melody).\nHOW IT HELPS: Flat lines (<15Hz) indicate 'Flat Affect' and clinical emotional numbing.", color=C_WARN, fontsize=16, labelpad=15)
    axes[2].set_facecolor('#0d1117'); axes[2].tick_params(colors=C_TEXT)

    # ROW 3: SILENCE
    axes[3].pie([100-silence_ratio, silence_ratio], labels=['Speech', 'Silence'], colors=[C_HL, C_BLUE], autopct='%1.1f%%', textprops={'color':"w", 'fontsize':18})
    axes[3].set_title("3. COGNITIVE LOAD ANALYSIS", color=C_TEXT, fontsize=22, fontweight='bold', pad=20)
    axes[3].set_xlabel("WHAT IS THIS: Ratio of unintended gaps. High ratio = Cognitive Distress.", color=C_WARN, fontsize=16, labelpad=15)

    # ROW 4: RADAR
    ax = plt.subplot(10, 1, 5, polar=True); ax.set_facecolor('#111827')
    r_labels = ['Sadness', 'Neutral', 'Calm', 'Energy', 'Happiness']
    r_vals = [cry_s*60, speech_s*10, silence_ratio/100, 0.4, laugh_s*20, cry_s*60]
    angles = np.linspace(0, 2*np.pi, len(r_labels)+1, endpoint=True)
    ax.fill(angles, r_vals, color=C_BLUE, alpha=0.4); ax.plot(angles, r_vals, color=C_BLUE, linewidth=2)
    ax.set_xticks(angles[:-1]); ax.set_xticklabels(r_labels, color=C_TEXT, fontsize=14)
    ax.set_title("4. EMOTION RADAR CLASSIFICATION", color=C_TEXT, fontsize=22, fontweight='bold', pad=40)

    # ROW 5: PRIVACY
    ax = axes[5]; S = librosa.feature.melspectrogram(y=y, sr=sr)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max), x_axis='time', y_axis='mel', sr=sr, ax=ax, cmap='magma')
    ax.set_title("5. PRIVACY SHIELD (MEL-SPECTROGRAM)", color=C_TEXT, fontsize=22, fontweight='bold', pad=20)
    ax.set_xlabel("PRIVACY: Our AI analyzes these frequency patterns, NOT the words. Safe & Private.", color=C_HL, fontsize=16, fontweight='bold', labelpad=15)

    # ROW 6: TEXTURE
    ax = axes[6]; ax.set_facecolor('#0d1117')
    ax.scatter(np.random.normal(jitter, 0.04, 100), np.random.normal(shimmer, 0.08, 100), color=C_WARN, alpha=0.6, s=150)
    ax.set_title("6. VOCAL TEXTURE (Micro-Instability)", color=C_TEXT, fontsize=22, fontweight='bold', pad=20)

    # --- ROW 7: GUARDIAN BEHAVIORAL LOGIC (UPDATED WITH TABLE) ---
    ax = axes[7]; t_ax = np.linspace(0, duration, scores.shape[0])
    ax.set_facecolor('#0d1117')

    # 1. Active Social Engagement (Speech)
    ax.plot(t_ax, scores[:, class_names.index('Speech')], color=C_HL, label="Social Engagement (Speech)", alpha=0.6)
    # 2. Lethargic Vocalization (Whispering)
    ax.plot(t_ax, scores[:, class_names.index('Whispering')], color=C_PURP, label="Lethargy (Mumble)", linewidth=2)
    # 3. Anxiety / Fatigue (Sigh + Breathing)
    ax.plot(t_ax, scores[:, class_names.index('Sigh')] + scores[:, class_names.index('Breathing')], color=C_WARN, label="Anxiety (Sighs)", linewidth=2)
    # 4. Immediate Crisis (Crying)
    ax.plot(t_ax, scores[:, class_names.index('Crying, sobbing')], color=C_CRIT, label="CRISIS (Crying)", linewidth=4)

    ax.set_title("7. GUARDIAN LOGIC (Behavioral Detections)", color=C_TEXT, fontsize=22, fontweight='bold', pad=20)
    ax.legend(loc='upper right', facecolor='#111', labelcolor='white', fontsize=12)
    ax.set_xlabel("WHAT IS THIS: Real-time detection of Lethargy (Mumble), Anxiety (Sighs), and Crisis (Crying).\nHOW IT HELPS: Maps clinical behaviors to timelines without recording words.", color=C_WARN, fontsize=16, labelpad=15)

    # ROW 8: TREND
    ax = axes[8]; ax.set_facecolor(C_BG)
    time_24 = np.linspace(0, 24, 100); act = np.abs(np.sin(time_24/3.5)*40) + 12
    ax.fill_between(time_24, act, color=C_HL, alpha=0.2); ax.plot(time_24, act, color=C_HL, linewidth=4)
    ax.set_title("8. GUARDIAN TREND (24h Social Battery)", color=C_TEXT, fontsize=22, fontweight='bold', pad=20)
    ax.set_xlim(0, 24); ax.tick_params(colors=C_TEXT)

    # --- ROW 9: THE OVERFLOW-PROOF RISK BOX (HARD CONTAINER) ---
    ax = axes[9]; ax.set_facecolor(C_BG)
    ax.axis('off')

    # 1. The Physical Container Box
    rect = patches.Rectangle((0.05, 0.05), 0.9, 0.9, linewidth=6, edgecolor=risk_col, facecolor='#111827', transform=ax.transAxes)
    ax.add_patch(rect)

    # 2. The Title
    ax.text(0.5, 0.82, "FINAL CLINICAL RISK ASSESSMENT", color=C_TEXT, fontsize=28, ha='center', fontweight='black', transform=ax.transAxes)

    # 3. The Big Score
    ax.text(0.5, 0.52, f"{risk_pct}%", color=risk_col, fontsize=120, ha='center', fontweight='black', transform=ax.transAxes)

    # 4. The Progress Bar
    ax.barh([0.38], [0.8], color='#222', height=0.05, align='center', transform=ax.transAxes, left=0.1)
    ax.barh([0.38], [0.8 * (risk_pct/100)], color=risk_col, height=0.05, align='center', transform=ax.transAxes, left=0.1)

    # 5. The Status Label
    ax.text(0.5, 0.18, f"STATUS: {risk_lvl}", color=risk_col, fontsize=38, ha='center', fontweight='bold',
            bbox=dict(facecolor='black', alpha=0.9, edgecolor=risk_col, boxstyle='round,pad=1.2'), transform=ax.transAxes)

    plt.show()

# --- RUN ---
uploaded = files.upload()
for filename in uploaded.keys():
    draw_medical_cockpit(filename)

KeyboardInterrupt: 

### **API**

In [5]:
import io
import tempfile
import shutil
import os
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import Response, HTMLResponse
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import librosa
import librosa.display
import parselmouth
from parselmouth.praat import call
import tensorflow as tf
import tensorflow_hub as hub
import csv
import warnings

# --- INITIAL SETUP AND GLOBAL RESOURCES ---
warnings.filterwarnings('ignore')

# --- MODERN DESIGN PALETTE (Kept from original) ---
C_BG = '#0B0F19'      # Deep Space Navy
C_HL = '#00E676'      # Neon Healthy Green
C_WARN = '#FFEA00'    # Neon Warning Yellow
C_CRIT = '#FF1744'    # Neon Critical Red
C_BLUE = '#2979FF'    # Electric Blue
C_PURP = '#D500F9'    # Lethargy Purple
C_TEXT = '#FFFFFF'

# Global variables for the model
yamnet_model = None
class_names = []

# --- FASTAPI APP ---
app = FastAPI(
    title="Guardian AI Vocal Analysis API",
    description="Analyzes vocal biometrics from an audio file using Librosa, Praat (Parselmouth), and YAMNet."
)

@app.on_event("startup")
async def startup_event():
    """Load the heavy YAMNet model once when the app starts."""
    global yamnet_model, class_names
    print("Waking up Guardian AI Engine...")
    try:
        # Suppress TF messages for cleaner startup
        os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
        tf.get_logger().setLevel('ERROR')

        yamnet_model = hub.load('https://tfhub.dev/google/yamnet/1')
        class_map_path = yamnet_model.class_map_path().numpy().decode('utf-8')
        class_names = [row['display_name'] for row in csv.DictReader(open(class_map_path))]
        print("Guardian AI Engine ready.")
    except Exception as e:
        print(f"CRITICAL ERROR: Failed to load YAMNet model: {e}")
        yamnet_model = None


# --- HELPER FUNCTIONS (Kept from original) ---

def get_vocal_texture_safe(snd: parselmouth.Sound):
    """Calculates Jitter and Shimmer safely."""
    try:
        pitch = call(snd, "To Pitch", 0.0, 75, 600)
        point_process = call([snd, pitch], "To PointProcess (cc)")
        jitter = call([snd, point_process], "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
        shimmer = call([snd, point_process], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
        if np.isnan(jitter) or jitter == 0: return 0.2, 1.0
        return jitter * 100, shimmer * 100
    except:
        return 0.15, 0.8

def calculate_modern_risk(f0_std, wpm, silence_ratio, cry_score, mumble_score, sigh_score, laugh_score):
    """Calculates risk based on the full Clinical Requirement Table."""
    score = 10 # Baseline
    score += np.clip((30 - f0_std) * 1.5, 0, 30)
    score += np.clip((130 - wpm) * 0.4, 0, 30)
    score += np.clip(silence_ratio * 0.5, 0, 20)
    score += np.clip(cry_score * 100, 0, 25)
    score += np.clip(mumble_score * 50, 0, 15)
    score += np.clip(sigh_score * 50, 0, 10)
    score -= np.clip(laugh_score * 50, 0, 20)

    final_pct = int(np.clip(score, 5, 100))

    if final_pct > 60: level, col = "CRITICAL ALERT", C_CRIT
    elif final_pct > 30: level, col = "MODERATE WARNING", C_WARN
    else: level, col = "STABLE / HEALTHY", C_HL
    return final_pct, level, col

def draw_medical_cockpit(file_path: str) -> bytes:
    """
    Processes audio, calculates metrics, and generates the Matplotlib plot as PNG bytes.
    Modified to be non-interactive and return bytes.
    """
    global yamnet_model, class_names

    if yamnet_model is None:
        raise RuntimeError("YAMNet model failed to load at startup. Check startup logs.")

    # --- DATA PROCESSING ---
    y, sr = librosa.load(file_path, sr=16000)
    duration = librosa.get_duration(y=y, sr=sr)
    snd = parselmouth.Sound(file_path)

    # Physics Metrics
    pitch = snd.to_pitch()
    f_vals = pitch.selected_array['frequency']
    f_vals[f_vals == 0] = np.nan
    f0_std = np.nanstd(f_vals)
    if np.isnan(f0_std): f0_std = 12.0

    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    peaks = librosa.util.peak_pick(onset_env, pre_max=3, post_max=3, pre_avg=3, post_avg=5, delta=0.5, wait=10)
    wpm = (len(peaks) / duration) * 60 if duration > 0 else 0

    intervals = librosa.effects.split(y, top_db=25)
    silence_duration = duration - (sum([i[1]-i[0] for i in intervals])/sr)
    silence_ratio = (silence_duration / duration) * 100 if duration > 0 else 0
    jitter, shimmer = get_vocal_texture_safe(snd)

    # AI YAMNet Scores
    scores, _, _ = yamnet_model(tf.cast(y, tf.float32))
    m_scores = np.mean(scores.numpy(), axis=0)

    def get_val(name):
        try:
            return m_scores[class_names.index(name)]
        except ValueError:
            return 0.0

    # --- GUARDIAN LABELS EXTRACTION ---
    speech_s = get_val('Speech')
    sigh_s = get_val('Sigh') + get_val('Breathing') # Combined Anxiety
    mumble_s = get_val('Whispering') # Proxy for Lethargy
    cry_s = get_val('Crying, sobbing') # Crisis
    laugh_s = get_val('Laughter') # Positive

    # Risk Calculation
    risk_pct, risk_lvl, risk_col = calculate_modern_risk(f0_std, wpm, silence_ratio, cry_s, mumble_s, sigh_s, laugh_s)

    # --- UI BUILDER (Modified for API Output) ---
    plt.ioff() # Turn off interactive mode
    fig, axes = plt.subplots(nrows=10, ncols=1, figsize=(18, 95), facecolor=C_BG)
    plt.subplots_adjust(hspace=0.9, bottom=0.05)

    # --- PLOT GENERATION (Original logic) ---

    # ROW 0: HEADER
    axes[0].set_facecolor('#111827')
    axes[0].text(0.5, 0.5, "GUARDIAN AI: BIOMETRIC SCAN", color=C_TEXT, fontsize=32, ha='center', fontweight='black')
    axes[0].axis('off')

    # ROW 1: SPEED
    axes[1].barh([0], [200], color='#222', height=0.3)
    axes[1].barh([0], [wpm], color=C_BLUE, height=0.3)
    axes[1].set_title(f"1. PSYCHOMOTOR SPEED ({int(wpm)} WPM)", color=C_TEXT, fontsize=22, fontweight='bold', pad=20)
    axes[1].set_xlabel("WHAT IS THIS: Measures 'Psychomotor Retardation' via speech frequency.\nHOW IT HELPS: Depression slows cognitive processing; lower WPM equals clinical lethargy.", color=C_WARN, fontsize=16, labelpad=15); axes[1].axis('off')

    # ROW 2: PROSODY
    axes[2].plot(pitch.xs(), f_vals, color=C_HL, linewidth=3)
    axes[2].set_title(f"2. PROSODY CONTOUR (SD: {f0_std:.1f}Hz)", color=C_TEXT, fontsize=22, fontweight='bold', pad=20)
    axes[2].set_xlabel("WHAT IS THIS: Tracks the fundamental frequency variance (Vocal Melody).\nHOW IT HELPS: Flat lines (<15Hz) indicate 'Flat Affect' and clinical emotional numbing.", color=C_WARN, fontsize=16, labelpad=15)
    axes[2].set_facecolor('#0d1117'); axes[2].tick_params(colors=C_TEXT)

    # ROW 3: SILENCE
    axes[3].pie([100-silence_ratio, silence_ratio], labels=['Speech', 'Silence'], colors=[C_HL, C_BLUE], autopct='%1.1f%%', textprops={'color':"w", 'fontsize':18})
    axes[3].set_title("3. COGNITIVE LOAD ANALYSIS", color=C_TEXT, fontsize=22, fontweight='bold', pad=20)
    axes[3].set_xlabel("WHAT IS THIS: Ratio of unintended gaps. High ratio = Cognitive Distress.", color=C_WARN, fontsize=16, labelpad=15)

    # ROW 4: RADAR (Requires manual handling of polar plot to replace the default axis)
    fig.delaxes(axes[4])
    ax_radar = fig.add_subplot(10, 1, 5, polar=True); ax_radar.set_facecolor('#111827')
    r_labels = ['Sadness', 'Neutral', 'Calm', 'Energy', 'Happiness']
    r_vals = [cry_s*60, speech_s*10, silence_ratio/100, 0.4, laugh_s*20]
    r_vals_closed = r_vals + [r_vals[0]]
    angles = np.linspace(0, 2*np.pi, len(r_labels), endpoint=False)
    angles_closed = np.concatenate((angles, [angles[0]]))

    ax_radar.fill(angles_closed, r_vals_closed, color=C_BLUE, alpha=0.4); ax_radar.plot(angles_closed, r_vals_closed, color=C_BLUE, linewidth=2)
    ax_radar.set_xticks(angles); ax_radar.set_xticklabels(r_labels, color=C_TEXT, fontsize=14)
    ax_radar.set_title("4. EMOTION RADAR CLASSIFICATION", color=C_TEXT, fontsize=22, fontweight='bold', pad=40)
    axes[4] = ax_radar # Store the new axis object

    # ROW 5: PRIVACY (Spectrogram)
    ax = axes[5]; S = librosa.feature.melspectrogram(y=y, sr=sr)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max), x_axis='time', y_axis='mel', sr=sr, ax=ax, cmap='magma')
    ax.set_title("5. PRIVACY SHIELD (MEL-SPECTROGRAM)", color=C_TEXT, fontsize=22, fontweight='bold', pad=20)
    ax.set_xlabel("PRIVACY: Our AI analyzes these frequency patterns, NOT the words. Safe & Private.", color=C_HL, fontsize=16, fontweight='bold', labelpad=15)

    # ROW 6: TEXTURE
    ax = axes[6]; ax.set_facecolor('#0d1117')
    ax.scatter(np.random.normal(jitter, 0.04, 100), np.random.normal(shimmer, 0.08, 100), color=C_WARN, alpha=0.6, s=150)
    ax.set_title("6. VOCAL TEXTURE (Micro-Instability)", color=C_TEXT, fontsize=22, fontweight='bold', pad=20)
    ax.tick_params(colors=C_TEXT) # Added for visibility

    # ROW 7: GUARDIAN BEHAVIORAL LOGIC
    ax = axes[7]; t_ax = np.linspace(0, duration, scores.shape[0])
    ax.set_facecolor('#0d1117')
    ax.plot(t_ax, scores[:, class_names.index('Speech')], color=C_HL, label="Social Engagement (Speech)", alpha=0.6)
    ax.plot(t_ax, scores[:, class_names.index('Whispering')], color=C_PURP, label="Lethargy (Mumble)", linewidth=2)
    ax.plot(t_ax, scores[:, class_names.index('Sigh')] + scores[:, class_names.index('Breathing')], color=C_WARN, label="Anxiety (Sighs)", linewidth=2)
    ax.plot(t_ax, scores[:, class_names.index('Crying, sobbing')], color=C_CRIT, label="CRISIS (Crying)", linewidth=4)
    ax.set_title("7. GUARDIAN LOGIC (Behavioral Detections)", color=C_TEXT, fontsize=22, fontweight='bold', pad=20)
    ax.legend(loc='upper right', facecolor='#111', labelcolor='white', fontsize=12)
    ax.set_xlabel("WHAT IS THIS: Real-time detection of Lethargy (Mumble), Anxiety (Sighs), and Crisis (Crying).\nHOW IT HELPS: Maps clinical behaviors to timelines without recording words.", color=C_WARN, fontsize=16, labelpad=15)
    ax.tick_params(colors=C_TEXT)

    # ROW 8: TREND
    ax = axes[8]; ax.set_facecolor(C_BG)
    time_24 = np.linspace(0, 24, 100); act = np.abs(np.sin(time_24/3.5)*40) + 12
    ax.fill_between(time_24, act, color=C_HL, alpha=0.2); ax.plot(time_24, act, color=C_HL, linewidth=4)
    ax.set_title("8. GUARDIAN TREND (24h Social Battery)", color=C_TEXT, fontsize=22, fontweight='bold', pad=20)
    ax.set_xlim(0, 24); ax.tick_params(colors=C_TEXT)

    # --- ROW 9: THE OVERFLOW-PROOF RISK BOX ---
    ax = axes[9]; ax.set_facecolor(C_BG); ax.axis('off')
    rect = patches.Rectangle((0.05, 0.05), 0.9, 0.9, linewidth=6, edgecolor=risk_col, facecolor='#111827', transform=ax.transAxes)
    ax.add_patch(rect)
    ax.text(0.5, 0.82, "FINAL CLINICAL RISK ASSESSMENT", color=C_TEXT, fontsize=28, ha='center', fontweight='black', transform=ax.transAxes)
    ax.text(0.5, 0.52, f"{risk_pct}%", color=risk_col, fontsize=120, ha='center', fontweight='black', transform=ax.transAxes)
    ax.barh([0.38], [0.8], color='#222', height=0.05, align='center', transform=ax.transAxes, left=0.1)
    ax.barh([0.38], [0.8 * (risk_pct/100)], color=risk_col, height=0.05, align='center', transform=ax.transAxes, left=0.1)
    ax.text(0.5, 0.18, f"STATUS: {risk_lvl}", color=risk_col, fontsize=38, ha='center', fontweight='bold',
            bbox=dict(facecolor='black', alpha=0.9, edgecolor=risk_col, boxstyle='round,pad=1.2'), transform=ax.transAxes)

    # --- RETURN IMAGE DATA AS BYTES ---
    buf = io.BytesIO()
    try:
        fig.savefig(buf, format='png', bbox_inches='tight', facecolor=C_BG)
    finally:
        plt.close(fig) # IMPORTANT: Frees up memory after plot generation

    return buf.getvalue()


# --- FASTAPI ENDPOINT ---

@app.post("/analyze_audio", response_class=Response, summary="Analyze Audio and Generate Cockpit Visualization")
async def analyze_audio_route(audio_file: UploadFile = File(..., description="Audio file (e.g., MP3, WAV) for vocal biometric analysis.")):
    """
    Accepts an audio file, analyzes vocal features, calculates a clinical risk score,
    and returns a massive multi-panel visualization as a PNG image.
    """

    if yamnet_model is None:
         raise HTTPException(status_code=503, detail="AI Model not ready. Please check server logs.")

    temp_file = None
    try:
        # 1. Save the uploaded file to a temporary location
        # This is required because parselmouth/librosa often expect a file path, not just file content
        ext = audio_file.filename.split('.')[-1] if '.' in audio_file.filename else 'wav'
        with tempfile.NamedTemporaryFile(delete=False, suffix=f".{ext}") as tmp:
            shutil.copyfileobj(audio_file.file, tmp)
            temp_file = tmp.name

        # 2. Process the audio and generate the image bytes
        image_bytes = draw_medical_cockpit(temp_file)

        # 3. Return the image as a response
        return Response(content=image_bytes, media_type="image/png")

    except HTTPException as e:
        raise e
    except Exception as e:
        print(f"Error: {e}")
        raise HTTPException(status_code=400, detail=f"Audio processing failed. Is the file a valid audio format? Error: {str(e)}")
    finally:
        # 4. Clean up the temporary file
        if temp_file and os.path.exists(temp_file):
            os.remove(temp_file)

# Optional: Simple root endpoint for documentation
@app.get("/")
async def root():
    return {"message": "Go to /docs to use the API endpoint: /analyze_audio"}